**API à utiliser :**


TIME FORMAT : Unix Milliseconds time format 

                        ACCOUNT V1 

Récupération PUUID : https://developer.riotgames.com/apis#account-v1/GET_getByRiotId 

        /riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}

                        MATCH V5 

Récupération MatchId : https://developer.riotgames.com/apis#match-v5/GET_getMatchIdsByPUUID

        /lol/match/v5/matches/by-puuid/{puuid}/ids

Récupération des informations des parties précédentes : https://developer.riotgames.com/apis#match-v5/GET_getMatch

        /lol/match/v5/matches/{matchId}

Récupération de la timeline de game : https://developer.riotgames.com/apis#match-v5/GET_getTimeline

        /lol/match/v5/matches/{matchId}/timeline

                        SPECTATOR V5

Récupération des informations Current Game : https://developer.riotgames.com/apis#spectator-v5/GET_getCurrentGameInfoByPuuid

        /lol/spectator/v5/active-games/by-summoner/{encryptedPUUID}

                        CHAMPION MASTERY V4

Récupération de la maitrise champion que tu veux : https://developer.riotgames.com/apis#champion-mastery-v4/GET_getChampionMasteryByPUUID

        /lol/champion-mastery/v4/champion-masteries/by-puuid/{encryptedPUUID}/by-champion/{championId}

Récupération des maitrises des "TOP" champions : https://developer.riotgames.com/apis#champion-mastery-v4/GET_getTopChampionMasteriesByPUUID

        /lol/champion-mastery/v4/champion-masteries/by-puuid/{encryptedPUUID}/top

**Importation des bibliothèques pour récupérer les données de l'API League Of Legend**

In [1]:
import requests
import pandas as pd

# La clé est valide pendant 24h : 14/01 15h30
clef = "RGAPI-32afb638-a1d3-4c0e-9332-a5c162c2a085"
region = "euw1"
gameName = "Vhorg"
tagLine = "EUW"

**RECUPERATION DU PUUID EN FONCTION DU NOM ET TAGLINE**

In [2]:
# Définir les paramètres de base
API_KEY = clef  # Remplacez par votre clé API
BASE_URL = "https://europe.api.riotgames.com"  # Remplacez <region> par la région appropriée (ex: americas, asia, europe)
GAME_NAME = gameName  # Remplacez par le gameName du joueur
TAG_LINE = tagLine  # Remplacez par le tagLine du joueur

def get_puuid(game_name, tag_line):
    """
    Récupère le PUUID d'un joueur donné via son Riot ID (gameName et tagLine).
    
    :param game_name: Nom d'utilisateur du joueur (gameName)
    :param tag_line: Tagline du joueur (#1234 sans le #)
    :return: Le PUUID du joueur, ou None en cas d'échec
    """
    endpoint = f"{BASE_URL}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    headers = {"X-Riot-Token": API_KEY}
    
    # Envoyer la requête
    response = requests.get(endpoint, headers=headers)
    
    # Gérer les erreurs potentielles
    if response.status_code == 200:
        data = response.json()
        return data.get("puuid")  # Récupérer le PUUID dans la réponse
    else:
        print(f"Erreur: {response.status_code} - {response.text}")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    puuid = get_puuid(GAME_NAME, TAG_LINE)
    if puuid:
        print("PUUID récupéré :", puuid)
    else:
        print("Impossible de récupérer le PUUID.")


PUUID récupéré : bnfzljX--PoTHkybKyJX_KTGQ2sL6eNZLiHpV99yoES06ZfaPZ9vnWWyRIVZgCFBH3sxJEZ9CW-N1A


**Récuperation des match_id de 5 Matchs précédents**

In [3]:
def get_match_ids(puuid, start=0, count=20):
    """
    Récupère les IDs des matchs associés à un PUUID donné.
    
    :param puuid: Le PUUID du joueur
    :param start: Index de début pour la pagination
    :param count: Nombre de matchs à récupérer
    :return: Liste des IDs des matchs, ou None en cas d'échec
    """
    base_url = "https://europe.api.riotgames.com" # Ne pas modifier
    endpoint = f"{base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids"
    headers = {"X-Riot-Token": API_KEY}
    params = {
        "start": start,
        "count": count
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # Retourne la liste des IDs
    else:
        print(f"Erreur lors de la récupération des match IDs : {response.status_code} - {response.text}")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    # Étape 1 : Récupérer le PUUID
    puuid = get_puuid(GAME_NAME, TAG_LINE)
    
    if puuid:
        print("PUUID récupéré :", puuid)
        
        # Étape 2 : Récupérer les match IDs
        match_ids = get_match_ids(puuid, start=0, count=5) # Count à changer si on veut récupérer plus de matchs
        if match_ids:
            print("Match IDs récupérés :", match_ids)
        else:
            print("Impossible de récupérer les match IDs.")
    else:
        print("Impossible de récupérer le PUUID.")

PUUID récupéré : bnfzljX--PoTHkybKyJX_KTGQ2sL6eNZLiHpV99yoES06ZfaPZ9vnWWyRIVZgCFBH3sxJEZ9CW-N1A
Match IDs récupérés : ['EUW1_7267496131', 'EUW1_7266829232', 'EUW1_7266788397', 'EUW1_7261413953', 'EUW1_7260108418']


**RECUPERATION DES INFOS DE JEUX**

In [ ]:

def get_match_details(match_id):
    """
    Récupère les informations détaillées d'un match donné par son matchId.
    
    :param match_id: L'ID unique du match
    :return: Les détails du match sous forme de dictionnaire, ou None en cas d'échec
    """
    base_url = "https://europe.api.riotgames.com"
    endpoint = f"{base_url}/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": API_KEY}
    
    response = requests.get(endpoint, headers=headers)
    
    if response.status_code == 200:
        return response.json()  # Retourne les détails du match
    else:
        print(f"Erreur lors de la récupération des détails du match : {response.status_code} - {response.text}")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    # Exemple d'IDs de matchs récupérés
    match_ids
    
    for match_id in match_ids:
        print(f"Récupération des détails pour le match ID : {match_id}")
        match_details = get_match_details(match_id)
        if match_details:
            print(f"Détails du match {match_id} :")
            print(match_details)
        else:
            print(f"Impossible de récupérer les détails pour le match {match_id}.")


In [60]:
# Exemple des données reçues
data = match_details

# Extraire les informations souhaitées
filtered_data = {
    "gameDuration": data["info"]["gameDuration"],
    "gameId": data["info"]["gameId"],
    "gameMode": data["info"]["gameMode"],
    "gameName": data["info"]["gameName"],
    "gameType": data["info"]["gameType"],
    "gameVersion": data["info"]["gameVersion"],
    "mapId": data["info"]["mapId"],
    "participants": []
}

for participant in data["info"]["participants"]:
    filtered_participant = {
        "participantID" : participant["participantId"], #str
        "puuid" : participant["puuid"], #str
        "championId": participant["championId"], #int
        "championName": participant["championName"], #str
        "champExperience" : participant["champExperience"], #int
        "champLevel" : participant["champLevel"], #int
        "deaths": participant["deaths"], #int
        "damagePerMinute": participant["challenges"]["damagePerMinute"], #float %
        "kills": participant.get("kills", 0),  # Par défaut à 0 si non présent
        "assists": participant["assists"], #int
        "goldEarned": participant["goldEarned"], #int
        "teamDamagePercentage": participant["challenges"]["teamDamagePercentage"], #float %
        "assistPing" : participant["assistMePings"], #int
        "assist" : participant["assists"], #int
        "assistStreak" : participant["challenges"]["12AssistStreakCount"], #int
        "healFromMap" : participant["challenges"]["HealFromMapSources"], #int
        "abilityUses" : participant["challenges"]["abilityUses"], #int
        "supportQuest" : participant["challenges"]["completeSupportQuestInTime"], #int
        "controlWardsPlaced" : participant["challenges"]["controlWardsPlaced"], #int
        "damageTakenOnTeam" : participant["challenges"]["damageTakenOnTeamPercentage"], #float %
        "deathsByEnemy" : participant["challenges"]["deathsByEnemyChamps"], #int
        "dodgeSkillsShotSmallWindow" : participant["challenges"]["dodgeSkillShotsSmallWindow"], #int
        "effectiveHealAndShielding" : participant["challenges"]["effectiveHealAndShielding"], #int
        "enemyChampionImmobilizations" : participant["challenges"]["enemyChampionImmobilizations"], #int
        "goldPerMinute" : participant["challenges"]["goldPerMinute"], #int
        "immobilizeAndKillWithAlly" : participant["challenges"]["immobilizeAndKillWithAlly"], #int
        "kda" : participant["challenges"]["kda"], #float
        "killParticipation" : participant["challenges"]["killParticipation"], #int
        "knockEnemyIntoTeamAndKill" : participant["challenges"]["knockEnemyIntoTeamAndKill"], #int
        "landSkillShotsEarlyGame" : participant["challenges"]["landSkillShotsEarlyGame"], #int
        "laneMinionsFirst10Minutes" : participant["challenges"]["laneMinionsFirst10Minutes"], #ADC #int
        "laningPhaseGoldExpAdvantage" : participant["challenges"]["laningPhaseGoldExpAdvantage"], #int
        "legendaryItemUsed" : participant["challenges"]["legendaryItemUsed"], #Liste d'items #int
        "maxLevelLeadLaneOpponent" : participant["challenges"]["maxLevelLeadLaneOpponent"], #int
        "pickKillWithAlly" : participant["challenges"]["pickKillWithAlly"], #int
        "saveAllyFromDeath": participant["challenges"]["saveAllyFromDeath"], #int
        "skillshotsDodged" : participant["challenges"]["skillshotsDodged"],  #int
        "skillshotsHit" : participant["challenges"]["skillshotsHit"], #int
        "stealthWardsPlaced" : participant["challenges"]["stealthWardsPlaced"], #int
        "survivedThreeImmobilizesInFight" : participant["challenges"]["survivedThreeImmobilizesInFight"], #int
        "teamBaronKills" : participant["challenges"]["teamBaronKills"], #int
        "teamElderDragonKills" : participant["challenges"]["teamElderDragonKills"], #int
        "teamRiftHeraldKills" : participant["challenges"]["teamRiftHeraldKills"], #int
        "twoWardsOneSweeperCount" : participant["challenges"]["twoWardsOneSweeperCount"], #int
        "visionScoreAdvantage" : participant["challenges"]["visionScoreAdvantageLaneOpponent"], #int
        "visionScorePerMinute" : participant["challenges"]["visionScorePerMinute"], #int
        "wardTakedowns" : participant["challenges"]["wardTakedowns"], #int
        "wardTakedownsBefore20" : participant["challenges"]["wardTakedownsBefore20M"], #int
        "wardsGuarded" : participant["challenges"]["wardsGuarded"], #int
        "consumablesPurchased" : participant["consumablesPurchased"], #int
        "damageSelfMitigated" : participant["damageSelfMitigated"], #int
        "dangerPings" : participant["dangerPings"], #int
        "detectorWardsPlaced" : participant["detectorWardsPlaced"], #int
        "enemyMissingPings": participant["enemyMissingPings"], #int
        "enemyVisionPings": participant["enemyVisionPings"], #int
        "firstBloodAssist": participant["firstBloodAssist"], #bool
        "firstBloodKill": participant["firstBloodKill"], #bool
        "firstTowerAssist": participant["firstTowerAssist"], #bool
        "firstTowerKill": participant["firstTowerKill"], #bool
        "getBackPings": participant["getBackPings"], #int
        "goldSpent": participant["goldSpent"], #int
        "holdPings": participant["holdPings"],#int
        "individualPosition": participant["individualPosition"], #TOP, MID, JUNGLE, ADC, SUPPORT #str
        "item0": participant["item0"], #Item ID #int
        "item1": participant["item1"], #Item ID #int
        "item2": participant["item2"], #Item ID #int
        "item3": participant["item3"], #Item ID #int
        "item4": participant["item4"], #Item ID #int
        "item5": participant["item5"], #Item ID #int
        "item6": participant["item6"], #Item ID #int
        "itemsPurchased" : participant["itemsPurchased"], #int
        "killingSprees" : participant["killingSprees"], #int
        "magicDamageDealt" : participant["magicDamageDealt"], #int
        "magicDamageDealtToChampions" : participant["magicDamageDealtToChampions"], #int
        "magicDamageTaken" : participant["magicDamageTaken"], #int
        "onMyWay" : participant["onMyWayPings"], #int
        "physicalDamageDealt" : participant["physicalDamageDealt"], #int
        "physicalDamageDealtToChampions" : participant["physicalDamageDealtToChampions"], #int
        "physicalDamageTaken" : participant["physicalDamageTaken"], #int
        "retreatPings" : participant["retreatPings"], #int
        "riotIdGameName" : participant["riotIdGameName"], 
        "riotIdTagline" : participant["riotIdTagline"], #str
        "sightWardsBoughtInGame" : participant["sightWardsBoughtInGame"], #int
        "spell1Casts" : participant["spell1Casts"],  #int
        "spell2Casts" : participant["spell2Casts"], #int
        "spell3Casts" : participant["spell3Casts"], #int
        "spell4Casts" : participant["spell4Casts"], #int
        "summoner1Casts" : participant["summoner1Casts"],  #int
        "summonerId" : participant["summonerId"], #int
        "summoner2Casts" : participant["summoner2Casts"],  #int
        "summoner2Id" : participant["summoner2Id"],  #int
        "summonerId" : participant["summonerId"],  #int
        "summonerLevel" : participant["summonerLevel"],  #int
        "summonerName" : participant["summonerName"], #str
        "teamId" : participant["teamId"], #int
        "teamPosition" : participant["teamPosition"], #str
        "timeCCingOthers" : participant["timeCCingOthers"], #int
        "totalDamageDealt" : participant["totalDamageDealt"], #int
        "totalDamageDealtToChampions" : participant["totalDamageDealtToChampions"],  #int
        "totalDamageShieldedOnTeammates" : participant["totalDamageShieldedOnTeammates"],  #int
        "totalDamageTaken" : participant["totalDamageTaken"], #int
        "totalHeal" : participant["totalHeal"], #int
        "totalHealsOnTeammates" : participant["totalHealsOnTeammates"], #int
        "totalTimeCCDealt" : participant["totalTimeCCDealt"], #int
        "totalTimeSpentDead" : participant["totalTimeSpentDead"], #int
        "totalUnitsHealed" : participant["totalUnitsHealed"], #int
        "trueDamageTaken" : participant["trueDamageTaken"], #int
        "turretsLost" : participant["turretsLost"], #int
        "visionScore" : participant["visionScore"], #int
        "visionWardsBoughtInGame" : participant["visionWardsBoughtInGame"],  #int
        "wardsKilled" : participant["wardsKilled"], #int
        "wardsPlaced" : participant["wardsPlaced"] #int
   }
    filtered_data["participants"].append(filtered_participant)


TRANSFORMATION DU FICHIER JSON EN DATA FRAME

In [107]:

import pandas as pd

# Transforme la liste de participants en un DataFrame
df_participants = pd.DataFrame(filtered_data["participants"])

# Transforme les informations de partie en un DataFrame 
df_game = pd.DataFrame(filtered_data)

df_game_utilisable = df_game[["gameDuration","gameId","gameMode","gameName","gameType","gameVersion", "mapId"]]
  # df_game_utilisable_1 = df_game.iloc[0,:-1] Equivalent à au dessus si on peut optimiser le code 

# Affiche un aperçu du DataFrame
display(df_participants)

,participantID,puuid,championId,championName,champExperience,champLevel,deaths,damagePerMinute,kills,assists,...,totalHealsOnTeammates,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,trueDamageTaken,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced
0,1,bnfzljX--PoTHkybKyJX_KTGQ2sL6eNZLiHpV99yoES06Z...,897,KSante,15059,16,6,787.597864,9,9,...,0,2022,212,1,1492,11,16,0,0,10
1,2,MU6i4XHtZ9t3UPEQNSjRdj6TAu-zk1hFlQa6lrMQgKUKs0...,35,Shaco,14454,15,6,687.535486,13,9,...,0,627,231,1,932,11,16,0,1,7
2,3,JpFWqAyQrhktSvXhXYHq_J8jiZXvjxX73NXS0kCji-5KIf...,61,Orianna,15355,16,10,699.112738,4,21,...,0,364,315,1,1018,11,16,1,0,11
3,4,VXiwPTQtP7iXUP0gv40PHHHpaEV6_dfQA1Rv6073l7aj2U...,21,MissFortune,14537,15,10,900.379352,8,8,...,0,175,277,1,1024,11,28,1,1,12
4,5,GUUCOH92UN6sKYxZ29w_tRvN-nLYDtSvxDnfm9WmIKWJdc...,223,TahmKench,11764,14,7,460.447744,2,9,...,0,187,225,1,3226,11,32,1,4,12
5,6,O5LgldvkUP0eaWqBzJsL1Ij1Nv2Dkk1zoOX64w4T8XoPL6...,24,Jax,17435,17,9,1043.779887,7,3,...,0,120,361,1,950,2,11,0,0,6
6,7,DzFWj7VbcDEZi9yijlIiYHOHZZMGkpHiltVeU6RTedobMp...,33,Rammus,12828,14,10,365.255518,3,11,...,238,600,297,4,1747,2,21,0,1,8
7,8,X7A_HfDBSTr_1XvPWdcX1o8H2J0oLXnJlscfsy7Vp-GWUZ...,99,Lux,17466,17,6,1638.856579,18,4,...,0,587,135,1,680,2,16,0,0,10
8,9,7Gj426VauYI-0eYRI-w0vWd4ay-6cWyMQW3-6QX5LjL9Gg...,67,Vayne,18240,17,3,700.689011,10,7,...,0,139,121,1,906,2,10,0,0,6
9,10,eRkEECm0BQgJsppI5za0J2Vce8Ny8Qy_LeN-GbjKX7rHfN...,12,Alistar,12190,14,8,327.989072,1,18,...,4957,295,163,5,1737,2,71,4,2,31


TIMELINE MATCH A VOIR POUR AMELIORER LE MODELE

In [ ]:
def get_match_details_timeline(match_id):
    """
    Récupère les informations détaillées d'un match donné par son matchId.
    
    :param match_id: L'ID unique du match
    :return: Les détails du match sous forme de dictionnaire, ou None en cas d'échec
    """
    base_url = "https://europe.api.riotgames.com"
    endpoint_timeline = f"{base_url}/lol/match/v5/matches/{match_id}/timeline"
    headers = {"X-Riot-Token": API_KEY}
    
    response_timeline = requests.get(endpoint_timeline, headers=headers)
    
    if response_timeline.status_code == 200:
        return response_timeline.json()  # Retourne les détails du match
    else:
        print(f"Erreur lors de la récupération des détails du match : {response_timeline.status_code} - {response_timeline.text}")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    # Exemple d'IDs de matchs récupérés
    match_ids
    
    for match_id in match_ids:
        print(f"Récupération des détails pour le match ID : {match_id}")
        match_details_timeline = get_match_details_timeline(match_id)
        if match_details_timeline:
            print(f"Détails du match {match_id} :")
            print(match_details_timeline)
        else:
            print(f"Impossible de récupérer les détails pour le match {match_id}.")
